# Google form analysis

Analysis of results extracted from Google forms in csv format.

## Table of Contents


[Preparation](#preparation)

[Constants](#constants)

[Functions](#functions)
   
   - [general purpose](#genpurpose)
   
   - [sessions and temporalities](#sessions)
   
   - [score](#score)
   
   - [checkpoint validation](#checkvalidation)
   
   - [p(answered question N | answered question P)](#condproba)

## Preparation
<a id=preparation />

In [ ]:
import re
%run "../Functions/2. Game sessions.ipynb"

# Constants
<a id=constants />

In [ ]:
# special user ids
userIDThatDidNotAnswer = '"001c95c6-8207-43dc-a51b-adf0c6e005d7"'

userID1AnswerEN = '"00dbbdca-d86c-4bc9-803c-0602e0153f68"'
userIDAnswersEN = '"5977184a-1be2-4725-9b48-f2782dc03efb"'
userID1ScoreEN = '"6b5d392d-b737-49ef-99af-e8c445ff6379"'
userIDScoresEN = '"5ecf601d-4eac-433e-8056-3a5b9eda0555"'

userID1AnswerFR = '"2734a37d-4ba5-454f-bf85-1f7b767138f6"'
userIDAnswersFR = '"01e85778-2903-447b-bbab-dd750564ee2d"'
userID1ScoreFR = '"3d733347-0313-441a-b77c-3e4046042a53"'
userIDScoresFR = '"58d22690-8604-41cf-a5b7-d71fb3b9ad5b"'

userIDAnswersENFR = '"a7936587-8b71-43b6-9c61-17b2c2b55de3"'

In [ ]:
#localplayerguidkey = 'Ne pas modifier - identifiant anonyme prérempli'
localplayerguidkey = 'Do not edit -  pre-filled anonymous ID'
localplayerguidindex = gform.columns.get_loc(localplayerguidkey)
localplayerguidindex

In [ ]:
firstEvaluationQuestionKey = 'In order to modify the abilities of the bacterium, you have to...'
firstEvaluationQuestionIndex = gform.columns.get_loc(firstEvaluationQuestionKey)
firstEvaluationQuestionIndex

In [ ]:
answersColumnNameStem = "answers"
correctionsColumnNameStem = "corrections"

# Functions
<a id=functions />

## general purpose
<a id=genpurpose />

In [ ]:
#GForm
#user id in format localplayerguid = '8d352896-a3f1-471c-8439-0f426df901c1'

#RedMetrics
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

# source: https://stackoverflow.com/questions/42047994/regex-how-to-find-a-guid-in-a-long-string
GUIDpattern = '[0-9a-f]{8}-[0-9a-f]{4}-[1-5][0-9a-f]{3}-[89ab][0-9a-f]{3}-[0-9a-f]{12}'
#GUIDpattern = '[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}'
GFormGUIDpattern = '^' + GUIDpattern + '$'
RedMetricsGUIDpattern = '^"' + GUIDpattern + '"$'

def getRedMetricsGUIDFormat( guid ):
    if(re.search(RedMetricsGUIDpattern, guid)):
        return guid
    elif(re.search(GFormGUIDpattern, guid)):
        return '"' + guid + '"'
    else:
        print("incorrect GUID: check 1. Google form analysis' getRedMetricsGUIDFormat")

def getGFormGUIDFormat( guid ):
    if(re.search(RedMetricsGUIDpattern, guid)):
        return guid.replace('"','')
    elif(re.search(GFormGUIDpattern, guid)):
        return guid
    else:
        print("incorrect GUID: check 1. Google form analysis' getGFormGUIDFormat")

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId ):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    _columnAnswers = answers.T
    
    if 0 != len(answers):
        _newColumns = []
        for column in _columnAnswers.columns:
            _newColumns.append(answersColumnNameStem + str(column))
        _columnAnswers.columns = _newColumns
    else:
        # user has never answered
        print("user " + str(userId) + " has never answered")
        
    return _columnAnswers

## sessions and temporalities
<a id=sessions />

In [ ]:
# when did the user answer the questionnaire? After playing, before playing, undefined?
answerTemporalities = ['before', 'after', 'undefined'];
def setAnswerTemporalities( _df ):
    _df['Temporality'] = _df['Timestamp'].copy()
    for _index in _df.index:
        _firstEventDate = getFirstEventDate(getRedMetricsGUIDFormat(_df.loc[_index,localplayerguidkey]))
        _df.loc[_index,'Temporality'] = \
        getTemporality(_df.loc[_index,'Timestamp'],_firstEventDate)

# when did the user answer the questionnaire? 
# After gameEventDate, before gameEventDate, undefined?
# answerDate is assumed to be a string coding for tz='Europe/Berlin' time
# gameEventDate is assumed to be UTC
def getTemporality( answerDate, gameEventDate ):    
    if(gameEventDate == pd.Timestamp.max.tz_localize('utc')):
        return answerTemporalities[2]
    
    tzAnswerDate = pd.to_datetime(answerDate).tz_localize('Europe/Berlin')        
    if(tzAnswerDate < gameEventDate):
        return answerTemporalities[0]
    elif (tzAnswerDate > gameEventDate):
        return answerTemporalities[1]
    else:
        return answerTemporalities[2]

## score
<a id=score />

In [ ]:
def getCorrections( _userId ):
    _columnAnswers = getAnswers( _userId )

    if 0 != len(_columnAnswers.columns):

        _questionsCount = len(_columnAnswers.values)

        for _columnName in _columnAnswers.columns:
            language = _columnAnswers.loc['Language',_columnName]
            if answersColumnNameStem in _columnName:
                _answerNumber = _columnName.replace(answersColumnNameStem,"")
                newCorrectionsColumnName = correctionsColumnNameStem + _answerNumber

                _columnAnswers[newCorrectionsColumnName] = _columnAnswers[_columnName]
                _columnAnswers[newCorrectionsColumnName] = pd.Series(np.full(_questionsCount, np.nan))

                for question in _columnAnswers[_columnName].index:
                    _correctAnswer = correctAnswers.loc[question,language]
                    if(len(_correctAnswer) > 0):
                        _columnAnswers.loc[question,newCorrectionsColumnName] =\
                                str(_columnAnswers.loc[question,_columnName]).startswith(_correctAnswer)

    else:
        # user has never answered
        print("can't give correct answers")
    return _columnAnswers

# edits in-place
# _corrections must be a dataframe full of corrections as produced above
def getBinarizedCorrections( _corrections ):
    for _columnName in _corrections.columns:
        for _index in _corrections[_columnName].index:
            if(True==_corrections.loc[_index,_columnName]):
                _corrections.loc[_index,_columnName] = 1
            elif (False==_corrections.loc[_index,_columnName]):
                _corrections.loc[_index,_columnName] = 0
    return _corrections

#function that returns the score from user id
def getScore( _userId ):
    _score = []
    if hasAnswered( _userId ):
        _columnAnswers = getCorrections(_userId)
        for _columnName in _columnAnswers.columns:
            # only work on corrected columns
            if correctionsColumnNameStem in _columnName:
                _counts = (columnAnswers[_columnName]).value_counts()
                _thisScore = 0
                if(True in _counts):
                    _thisScore = _counts[True]
                _score.append(_thisScore)
    else:
        print("user " + str(_userId) + " has never answered")
    return _score

## checkpoint validation
<a id=checkvalidation />

In [ ]:
#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    _validatedCheckpoints = []
    
    if hasAnswered( userId ):
        _columnAnswers = getCorrections( userId)
        
        for _columnName in _columnAnswers.columns:
            # only work on corrected columns
            if correctionsColumnNameStem in _columnName:        
                _questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))

                for _index in range(0, len(_questionnaireValidatedCheckpointsPerQuestion)):
                    if _columnAnswers[_columnName][_index]==True:
                        _questionnaireValidatedCheckpointsPerQuestion[_index] = checkpointQuestionMatching['checkpoint'][_index]
                    else:
                        _questionnaireValidatedCheckpointsPerQuestion[_index] = ''

                _questionnaireValidatedCheckpoints = _questionnaireValidatedCheckpointsPerQuestion.unique()
                _questionnaireValidatedCheckpoints = _questionnaireValidatedCheckpoints[_questionnaireValidatedCheckpoints!='']
                _questionnaireValidatedCheckpoints = pd.Series(_questionnaireValidatedCheckpoints)
                _questionnaireValidatedCheckpoints = _questionnaireValidatedCheckpoints.sort_values()
                _questionnaireValidatedCheckpoints.index = range(0, len(_questionnaireValidatedCheckpoints))
                
                _validatedCheckpoints.append(_questionnaireValidatedCheckpoints) 
    else:
        print("user " + str(userId) + " has never answered")
    return pd.Series(_validatedCheckpoints)

def getValidatedCheckpointsCounts( userId ):
    _validatedCheckpoints = getValidatedCheckpoints(userId)
    _counts = []
    for checkpointsList in _validatedCheckpoints:
        _counts.append(len(checkpointsList))
    return _counts

def getNonValidated( checkpoints ):
    _validationLists = []
    
    if 0!=len(checkpoints):
        for _validation in checkpoints:
            _result = pd.Series(np.setdiff1d(validableCheckpoints.values, _validation.values))
            _result = _result[_result != '']
            _result.index = range(0, len(_result))
            _validationLists.append(_result)
        return pd.Series(_validationLists)
    else:
        return validableCheckpoints

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)

def getNonValidatedCheckpointsCounts( userId ):
    _nonValidatedCheckpoints = getNonValidatedCheckpoints(userId)
    _counts = []
    for checkpointsList in _nonValidatedCheckpoints:
        _counts.append(len(checkpointsList))
    return _counts

## p(answered question N | answered question P)
<a id=condproba />

In [ ]:
# returns all rows of Google form's answers that answered an element of the array 'choice' for question number 'questionIndex'
def getAllAnswerRows(questionIndex, choice):
    return gform[gform.iloc[:, questionIndex].isin(choice)]

def getPercentCorrectPerColumn(_df):
    _count = len(_df)
    _percents = pd.Series(np.full(len(_df.columns), np.nan), index=_df.columns)
    for _rowIndex in _df.index:
        _language = _df['Language'][_rowIndex]
        for _columnName in _df.columns:
            _columnIndex = _df.columns.get_loc(_columnName)
            if ((_columnIndex >= firstEvaluationQuestionIndex) \
                and (_columnIndex < len(_df.columns)-3)):
                if(str(_df[_columnName][_rowIndex]).startswith(str(correctAnswers[_language][_columnIndex]))):
                    if (np.isnan(_percents[_columnName])):
                        _percents[_columnName] = 1;
                    else:
                        _percents[_columnName] = _percents[_columnName]+1
                else:
                    if (np.isnan(_percents[_columnName])):
                        _percents[_columnName] = 0;
                
    _percents = _percents/_count
    _percents['Count'] = _count
    return _percents

def getPercentCorrectKnowingAnswer(questionIndex, choice):
    _answerRows = getAllAnswerRows(questionIndex, choice);
    return getPercentCorrectPerColumn(_answerRows)